# Jaist実験環境構築

## Gatling編
```s
$ yum update -y
$ yum install -y java-11-openjdk 
$ wget https://repo1.maven.org/maven2/io/gatling/highcharts/gatling-charts-highcharts-bundle/3.3.0/gatling-charts-highcharts-bundle-3.3.0-bundle.zip
```

### シミュレーションファイル
```s

   
package testing.ramp2const

import java.util.UUID.randomUUID
import io.gatling.core.Predef._
import io.gatling.core.structure.ScenarioBuilder
import io.gatling.http.Predef._

class GetRampToConstSimulation extends Simulation {

  private val USERS = java.lang.Double.valueOf(sys.env.getOrElse("USERS", "50"))
  private val RAMP_DURATION = java.lang.Integer.valueOf(sys.env.getOrElse("RAMP_DURATION", "10"))
  private val CONST_DURATION = java.lang.Integer.valueOf(sys.env.getOrElse("CONST_DURATION", "10"))
  private val ENDPOINT = sys.env.getOrElse("ENDPOINT", "")
  private val HTTP_PATH = sys.env.getOrElse("HTTP_PATH", "/")

  private val scn: ScenarioBuilder = scenario("Load Testing")
      .exec(
        http("initiation")
          .get(ENDPOINT + HTTP_PATH)
          .check(status.is(200))
      )
      .pause(1)

  setUp(scn.inject(
      rampUsersPerSec(1) to (USERS) during (RAMP_DURATION seconds),
      constantUsersPerSec(USERS) during(CONST_DURATION seconds),
      rampUsersPerSec(USERS) to (1) during (RAMP_DURATION seconds)
  )
    .protocols(http
      .acceptHeader("text/html,application/json,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8")
    )
  )
}
```

## Nginx編
```s
$ yum update -y
$ cat << 'EOT' >> /etc/yum.repos.d/nginx.repo
[nginx-stable]
name=nginx stable repo
baseurl=http://nginx.org/packages/centos/$releasever/$basearch/
gpgcheck=1
enabled=1
gpgkey=https://nginx.org/keys/nginx_signing.key
module_hotfixes=true
EOT
$ yum install nginx -y

```

